In [ ]:
%load_ext autoreload
%autoreload 2
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/6/7 15:17
# @Author  : Wang Yujia
# @File    : mlp.ipynb
# @Description :
# @TODO:

# 0. what for
1. loss部分逻辑重写，添加`no grad`
2. 添加hooks
3. 使用pad_seq函数取代自己写的pad函数
4. 效果：可以train虽然loss不下降，不会出现NaN的问题

# 1. Preparations
## 1.1 global settings

In [ ]:
import pandas as pd
import numpy as np
import math
import random
import torch.utils.data
from mydataset import *
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
from visdom import Visdom
from graphviz import Digraph
from torchviz import make_dot
from torch.nn.utils.rnn import pad_sequence
import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import geomloss
import time

import scipy
from torch.nn import KLDivLoss
from torch.autograd.gradcheck import gradcheck
from config import bcolors

In [ ]:
from config import DefaultConfig
opt = DefaultConfig()

# nums of Gaussian kernels
N_gaussians = opt.N_gaussians

# dataset划分
batch_size = opt.batch_size
train_pct = opt.train_pct
vali_pct = opt.vali_pct
test_pct = opt.test_pct

# train and optim.
learning_rate = opt.lr
lr_for_mu = opt.lr_for_mu   # 给mu单独设置learning rate

total_train_step = 0
total_test_step = 0

MIN_LOSS = opt.MIN_LOSS
SAFETY = opt.SAFETY

# Range【一般用不到】
a = 0
b = 255

# Training data的粒度，画图会使用到
SCALE = opt.SCALE
# Target data是target_5时
TARGET = opt.TARGET


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

## 1.2 the data path


In [355]:
# Training data
train_path = opt.train_path

# Target data
target_path = opt.target_path

# data keys
data_key_path = opt.data_key_path

# NLL metric
NLL_metric_path = opt.NLL_metric_path

# Net path
net_root_path = opt.net_root_path

# 2. Dataset and Dataloader
1. DataLoader中的`shuffer=True`表示在每一次epoch中都打乱所有数据的顺序，然后以batch为单位从头到尾按顺序取用数据。这样的结果就是不同epoch中的数据都是乱序的,设置随机种子的作用就是让你的每一次训练都乱的一样，

## 2.1 Dataset and spliting


- 设置随机数种子

In [ ]:
def setup_seed(seed):
    torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # torch.backends.cudnn.deterministic = True

setup_seed(7)

- 读取data

In [356]:
dataset = myDataset(train_path, target_path, data_key_path, NLL_metric_path)

- 产生index的乱序排列

In [285]:
shuffled_indices = np.random.permutation(dataset.__len__())

train_idx = shuffled_indices[:int(train_pct*dataset.__len__())]
# train_idx = shuffled_indices
tmp = int((train_pct+vali_pct)*dataset.__len__())
val_idx = shuffled_indices[int(train_pct*dataset.__len__()):tmp]

test_idx = shuffled_indices[tmp:]
print(train_idx)

[ 873   39  223  511  533  357  977  420 1167   28 1011  334  567 1112
 1135   75  921  621  194  745 1183  788 1190 1158  358 1104  220  155
 1083  508  665 1119  553  811   45  796  562 1110  340  205 1111  474
  514  664  348  624 1047  639  392  181  479  605  739  887  398  766
 1155  647  699  993 1082  891   47  797 1100  760  500  686  960  692
  756  422  225  264  932  493  893  186  774  480  892  670  844  466
  729  814 1172  593  258  329  307  262  663  527  888  973   87  366
   73  273  532  784  417  890 1120  897   95  182  695  702    6 1060
  124  694  566  862  630 1043   81  109  711  865  156  853   62  408
 1191  315  325  445  250 1149  294 1194  432  849 1165  313  515  171
  687  939  486  478  183  608  923  375  369   91  477 1044  596  913
  539  941  165  196  556  359  604  989  794  975  700  388  518  943
  980  909  583  530 1092   82  163  991  352  656  396  571  875  433
  232  117  520  113  162  345  954  856  969   98 1089 1166  789  248
 1097 

- 根据这个乱序排列抽取dataset

In [286]:
# 保存idx
np.save('shuffled_indices',shuffled_indices)

## 2.2 Dataloader and collating
1. 主要是对label数据进行collate
    - 按照batch中的最大target data长度进行padding，padding with 0
2. 返回的结果多一个batch dim,比如下面的`5`
    - After collating:
        - `torch.Size([5, 3, 300]),torch.Size([5, 87, 2])`
        - `87`是最长的targets data长度

In [ ]:
def test_pad_sequence():
    seq1 = torch.tensor([[ 2., 0.04761905], [3., 0.14285714], [4., 0.04761905]])
    seq2 = torch.tensor([[ 1., 0.04761905]])
    seq3 = torch.tensor([[ 3., 0.14285714], [4., 0.04761905]])
    ls = list((seq1,seq2,seq3))
    ls_length = torch.tensor([3,1,2])
    ans = pad_sequence(ls,batch_first=True)
    print(ans)
    # seq_pak = pack_padded_sequence(ans,ls_length,batch_first=True,enforce_sorted=False)
    # seq_unpacked, lens_unpacked = pad_packed_sequence(seq_pak, batch_first=True)
    # lens_unpacked

### 2.2.1 No rescale

In [ ]:
def my_collate_fn(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []
    target_list = []
    setting_list = []
    metric_list = []

    data_len = len(data)      # 读进来的data batch的大小

    batch = 0


    while batch < data_len:
        # print("shape:",data[batch][0].shape) shape: (3, 300)

        # 所有GT model
        data_list.append(torch.tensor(data[batch][0]))

        target_list.append(torch.tensor(data[batch][1]))
        setting_list.append(torch.tensor(data[batch][2]))
        metric_list.append(torch.tensor(data[batch][3]))
        batch += 1

    # Pad target data with zeros
    target_padded = pad_sequence(target_list,batch_first=True)
    target_tensor = target_padded.float()

    data_tensor = torch.stack(data_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_tensor, setting_tensor, metric_tensor

### 2.2.2 Rescale

In [ ]:
def my_collate_fn_rescale(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []
    target_list = []
    data_len = len(data)      # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        data_df = pd.DataFrame(data[batch][0])
        data_GT = data_df.iloc[0:2,:]

        # Get the max/min in each chanel and broadcast it by hand
        data_max = np.tile(pd.DataFrame(np.max(data_GT,axis=1)), (1, data_GT.shape[1]))
        data_min = np.tile(pd.DataFrame(np.min(data_GT,axis=1)), (1, data_GT.shape[1]))
        # Rescale
        data_rescaled = a + (data_GT-data_min)*(b-a) / (data_max - data_min)
        data_df.iloc[0:2,:] = data_rescaled

        data_list.append(torch.tensor(np.array(data_df,dtype=float)))
        # print("After scale:",data_df)
        target_list.append(torch.tensor(data[batch][1]))
        batch += 1

    # Pad target data with zeros
    target_padded = pad_sequence(target_list,batch_first=True)

    data_tensor = torch.stack(data_list).float()
    target_tensor = target_padded.float()

    return data_tensor, target_tensor

### 2.2.3 log1p

In [ ]:
def my_collate_fn_log1p(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []
    target_list = []
    data_len = len(data)      # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        data_df = pd.DataFrame(data[batch][0])
        data_GT = data_df.iloc[0:2,:]

        data_log1p = -np.log(data_GT+MIN_LOSS)
        data_df.iloc[0:2,:] = data_log1p

        data_list.append(torch.tensor(np.array(data_df,dtype=float)))
        target_list.append(torch.tensor(data[batch][1]))
        batch += 1

    # Pad target data with zeros
    target_padded = pad_sequence(target_list,batch_first=True)

    data_tensor = torch.stack(data_list).float()
    target_tensor = target_padded.float()

    return data_tensor, target_tensor

### 2.2.4 DataLoader

In [357]:
train_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(train_idx), collate_fn = my_collate_fn)

val_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(val_idx),collate_fn = my_collate_fn)

# 注意test_loader的batch size
test_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(test_idx),collate_fn = my_collate_fn)

# 3. The Net and Init
1. BatchNorm1d: The mean and std are calculated per-dimension over the mini-batches
2.

In [ ]:
### BatchNorm2d测试
def test_BN():
    m = nn.BatchNorm2d(3, affine=False)  # affine: With Learnable Parameters or not
    print('m:', m)
    # The mean and std are calculated per-dimension over the mini-batches
    input = torch.tensor([
        [[1.,2.,3.,4.],[1.,2.,3.,4.],[-1.,-2.,-3.,-4.]],
        [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]]
    ], requires_grad=True)

    print('input:', input.shape)
    input = input.unsqueeze(dim=2)
    print('input:', input.shape)
    output = m(input) # 归一化
    print('output:', output.shape)
    print('output:', output)

In [ ]:
### 复现batchNorm2d在input shape为3维的情况
input = torch.tensor([[[1.,2.,3.,4.]],[[0.,0.,0.,0.]]])
# print(input.shape)
# torch.mean(input),torch.var(input,unbiased = False)
(input-torch.mean(input))/ torch.sqrt(torch.var(input,unbiased = False))

## 3.1 Init, Printer, Hook
- 设置网络初始权重: 不太work

In [ ]:
class model_param_init(nn.Module):
    def __init__(self, model):
        super().__init__()
        assert isinstance(model, nn.Module), 'model not a class nn.Module'
        self.net = model
        self.initParam()

    def initParam(self):
        for param in self.net.parameters():
            # nn.init.zeros_(param)
            # nn.init.ones_(param)
            # nn.init.normal_(param, mean=0, std=1)
            # nn.init.uniform_(param, a=0, b=1)
            # nn.init.constant_(param, val=1)   # 将所有权重初始化为1
            # nn.init.eye_(param)  # 只能将二维的tensor初始化为单位矩阵
            # nn.init.xavier_uniform_(param, gain=1)  # Glorot初始化  得到的张量是从-a——a中采用的
            # nn.init.xavier_normal_(param, gain=1)   # 得到的张量是从0-std采样的
            nn.init.kaiming_normal_(param, a=0, mode='fan_in', nonlinearity='relu') # he初始化方法
            # nn.init.kaiming_uniform_(param)

- print网络每层结构

In [ ]:
DEBUG = False
class PrintLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        if(DEBUG):
            print("This layer: ")
            print(x)      #print(x.shape)
        return x

- hook_backward_fn: 输入端的grad，输出端的grad，这里称呼的**输入与输出是站在前向传播的角度的**。如果模块有多个输入与输出的话， 其`grad_input`和`grad_output`可以是tuple类型。
- 与forward不同的是，backward传播的时候，**不仅反向传递input和output的grad，还会传递模块Parameter的grad**：
    - 比如fc模块，其`grad_input`是一个三元组的tuple，（对bias的梯度，对输入的梯度，对w的梯度）；
    - conv模块`grad_input`也是一个三元组tuple，为（对输入的梯度，对w的梯度，对bias的梯度

In [ ]:
# hook functions have to take these 3 input
def hook_forward_fn(module, input, output):
    print("It's forward: ")
    print(f"module: {module}")
    print(f"input: {input}")
    print(f"output: {output}")
    print("="*20)

def hook_backward_fn(module, grad_input, grad_output):
    print("It's backward: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

def hook_backward_fn_pi(module, grad_input, grad_output):
    print("It's backward in pi: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

def hook_backward_fn_sigma(module, grad_input, grad_output):
    print("It's backward in sigma: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

def hook_backward_fn_mu(module, grad_input, grad_output):
    print("It's backward in mu: ")
    print(f"module: {module}")
    print(f"grad_input: {grad_input}")
    print(f"grad_output: {grad_output}")
    print("="*20)

## 3.2 Conv结构-1


In [ ]:
# # Not Sequential
# class MLP_1_1(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff1 = nn.BatchNorm2d(num_features=3,affine=True)
#         self.BN_aff2 = nn.BatchNorm2d(num_features=3,affine=True)
#         self.BN_aff3 = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.3)
#
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#         self.ac_func = nn.Softplus()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,5), stride=(1,5), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#         self.linear1 = nn.Linear(30, 9)
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(9, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(9, n_gaussians)
#         self.z_sigma = nn.Linear(9, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.conv1(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.BN_aff1(x)
#         x = self.conv2(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.BN_aff2(x)
#         x = self.conv3(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.flatten(x)
#         x = self.linear1(x)
#         x = self.ac_func(self.drop(x))
#
#         pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         mu = torch.clamp(mu,1e-4)
#
#         sigma = torch.exp(self.z_sigma(x))
#         sigma = torch.clamp(sigma,1e-4)
#
#         return pi, mu, sigma

## 3.3 Conv结构-2
1. 换了一下conv的结构

In [ ]:
# 参数量747
class MLP_1_2(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=False)
        self.BN2 = nn.BatchNorm2d(num_features=3,affine=True)
        self.BN3 = nn.BatchNorm2d(num_features=6,affine=True)
        self.drop = nn.Dropout(0.3)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(1,6), stride=(1,3), padding=0,bias=False)
        self.ac_func = nn.Softplus()
        # self.ac_func = nn.ReLU()

        # 3, 3, 99
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(3,6), stride=6, padding=0,bias=False)
        # 6,1,33

        self.conv3 = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(1,3), stride=3, padding=0,bias=False)

        self.flatten = nn.Flatten()
        #self.linear1 = nn.Linear(48, 18)

        #无clip
        self.z_pi = nn.Sequential(
            nn.Linear(30, n_gaussians),
            nn.Softmax(dim=1)
        )

        # 有clip
        # self.z_pi = nn.Linear(30, n_gaussians)
        self.z_mu = nn.Linear(30, n_gaussians)
        self.z_sigma = nn.Linear(30, n_gaussians)

    def forward(self, x):
        x = self.BN1(x)
        # 加一个height维度

        x = torch.unsqueeze(x,dim=1)
        x = self.conv1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        x = self.conv2(x)
        x = self.BN3(x)
        x = self.ac_func(self.drop(x))

        # x = torch.unsqueeze(x,dim=2)
        x = self.conv3(x)
        x = self.ac_func(self.drop(x))

        x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.ac_func(self.drop(x))
        mu = self.z_mu(x)
        pi = self.z_pi(x)

        sigma = torch.exp(self.z_sigma(x))
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi,mu,sigma

## 3.4 MLP结构
### 3.4.1 4层

In [ ]:
# 参数量304,000+
# 4层
class MLP_1_3(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=300,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=100,affine=True)
        self.BN4 = nn.BatchNorm1d(num_features=30,affine=True)
        self.BN5 = nn.BatchNorm1d(num_features=12,affine=True)
        self.BN6 = nn.BatchNorm1d(num_features=12,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(900, 300)
        self.linear2 = nn.Linear(300, 100)
        self.linear3 = nn.Linear(100, 30)
        self.linear4 = nn.Linear(30, 12)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear2(x)
        x = self.BN3(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear3(x)
        x = self.BN4(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear4(x)
        x = self.BN5(x)
        x = self.ac_func3(self.drop(x))

        x = self.BN6(x)
        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        # sigma = torch.exp(self.z_sigma(x))
        sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

### 3.4.2 2层

In [ ]:
# 参数量91,000+
# 2层MLP+MDN
class MLP_1_4(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=100,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=12,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(900, 100)
        self.linear2 = nn.Linear(100, 12)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear2(x)
        x = self.BN3(x)
        x = self.ac_func3(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)

        sigma = torch.exp(self.z_sigma(x))
        # sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        return pi, mu, sigma

### 3.4.3 1层

In [280]:
# 参数量8233
# 1层MLP+MDN
class MLP_1_5(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=9,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(900, 9)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(9, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(9, n_gaussians)
        self.z_sigma = nn.Linear(9, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func3(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)

        sigma = torch.exp(self.z_sigma(x))
        # sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        return pi, mu, sigma

## 3.5 (3,100)的结构
1. 适配(3,100)的training data长度而不是(3,300)

In [ ]:
# # 参数量：498
# class MLP_2_1(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff1 = nn.BatchNorm2d(num_features=3,affine=True)
#         self.BN_aff2 = nn.BatchNorm2d(num_features=3,affine=True)
#         self.BN_aff3 = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.3)
#
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#         # self.ac_func = nn.Softplus()
#         # self.ac_func = nn.Sigmoid()
#         self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#         # self.linear1 = nn.Linear(30, 9)
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(48, n_gaussians),  # 30个params要learn
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(48, n_gaussians)
#         self.z_sigma = nn.Linear(48, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.conv1(x)
#         x = self.BN_aff1(x)
#         x = self.ac_func(self.drop(x))
#
#         # x = self.BN_aff2(x)
#         x = self.conv2(x)
#         x = self.BN_aff2(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.flatten(x)
#
#         pi = self.z_pi(torch.clamp(x,1e-3))
#         # pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         # sigma = torch.exp(self.z_sigma(x))
#         sigma = F.elu(self.z_sigma(x))+1
#
#         # pi.retain_grad()
#         # mu.retain_grad()
#         # sigma.retain_grad()
#         # print("grad:")
#         # print("pi.grad:",pi.grad)
#         # print("mu.grad:",mu.grad)
#         # print("sigma.grad:",sigma.grad)
#
#         # return x
#         return pi, mu, sigma

In [ ]:
# # 参数量：228
# class MLP_2_2(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.2)
#
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#         # self.ac_func = nn.Softplus()
#         self.ac_func = nn.LeakyReLU()
#         self.ac_func2 = nn.ReLU()
#
#         # self.ac_func = nn.Sigmoid()
#         # self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(15, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(15, n_gaussians)
#         self.z_sigma = nn.Linear(15, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.conv1(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.BN_aff(x)
#         x = self.conv2(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.BN_aff(x)
#         x = self.conv3(x)
#         x = self.ac_func(self.drop(x))
#         #
#         x = self.flatten(x)
#
#         pi = self.z_pi(torch.clamp(x,1e-3))
#         # pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         sigma = torch.exp(self.z_sigma(x))
#
#         # return x
#         return pi, mu, sigma

- MLP结构+长度=100

In [ ]:

class MLP_2_3(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=3,affine=False)
        self.BN2 = nn.BatchNorm1d(num_features=1,affine=False)
        self.drop = nn.Dropout(0.5)

        self.linear1 = nn.Linear(300, 100)
        self.linear2 = nn.Linear(100, 30)
        self.linear3 = nn.Linear(30, 12)

        self.ac_func = nn.Softplus()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.ac_func(self.drop(x))

        # x = self.BN2(x)
        x = self.linear2(x)
        x = self.ac_func(self.drop(x))

        # x = self.BN2(x)
        x = self.linear3(x)
        x = self.ac_func(self.drop(x))

        # pi = self.z_pi(x)
        pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        # sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.exp(self.z_sigma(x))

        # return x
        return pi, mu, sigma

## 3.6 (3,60)的结构
1. 适配(3,60)的training data长度而不是(3,300)

In [ ]:
# # 参数量：174
# class MLP_3_1(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.2)
#
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#         # self.ac_func = nn.Softplus()
#         self.ac_func = nn.LeakyReLU()
#         self.ac_func2 = nn.ReLU()
#
#         # self.ac_func = nn.Sigmoid()
#         # self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#         self.linear = nn.Linear(24, 9)
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(9, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(9, n_gaussians)
#         self.z_sigma = nn.Linear(9, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.conv1(x)
#         x = self.ac_func(self.drop(x))      # 3, 1, 20
#
#         x = self.BN_aff(x)
#         x = self.conv2(x)
#         x = self.ac_func(self.drop(x))      # 3, 1, 10
#
#         x = self.BN_aff(x)
#         x = self.conv3(x)
#         x = self.ac_func(self.drop(x))      # 3, 1, 8
#
#         x = self.flatten(x)
#         # print(x.shape)
#         # x = self.linear(x)
#         #
#         pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         sigma = torch.exp(self.z_sigma(x))
#
#         # return x
#         return pi, mu, sigma

- MLP

In [ ]:
# # 参数量：5799
# class MLP_3_2(nn.Module):
#     # code->generate->override methods
#     def __init__(self, n_gaussians) -> None:
#         super().__init__()
#         self.BN = nn.BatchNorm2d(num_features=3,affine=False)
#         self.BN_aff = nn.BatchNorm2d(num_features=3,affine=True)
#         self.drop = nn.Dropout(0.2)
#
#         self.linear1 = nn.Linear(180,30)
#         self.linear2 = nn.Linear(30,9)
#         # self.ac_func = nn.Softplus()
#         self.ac_func = nn.LeakyReLU()
#         self.ac_func2 = nn.ReLU()
#
#         # self.ac_func = nn.Sigmoid()
#         # self.ac_func = nn.Tanh()
#
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,2), stride=(1,2), padding=0,bias=False)
#
#         self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(1,3), stride=(1,3), padding=0,bias=False)
#
#         self.flatten = nn.Flatten()
#         self.linear = nn.Linear(24, 9)
#
#         self.z_pi = nn.Sequential(
#             nn.Linear(9, n_gaussians),
#             nn.Softmax(dim=1)
#         )
#
#         self.z_mu = nn.Linear(9, n_gaussians)
#         self.z_sigma = nn.Linear(9, n_gaussians)
#
#     def forward(self, x):
#         # 加一个height维度
#         x = torch.unsqueeze(x,dim=2)
#         x = self.BN(x)
#         x = self.flatten(x)
#
#         x = self.linear1(x)
#         x = self.ac_func(self.drop(x))
#
#         x = self.linear2(x)
#         x = self.ac_func(self.drop(x))
#
#         pi = self.z_pi(x)
#         mu = self.z_mu(x)
#         sigma = torch.exp(self.z_sigma(x))
#
#         # return x
#         return pi, mu, sigma

In [ ]:
input = torch.tensor([
    [[1.,2.,3.,4.],[1.,2.,3.,4.],[-1.,-2.,-3.,-4.]],
    [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]]
], requires_grad=True)
input = input.unsqueeze(dim=2)
print(f"input's shape is {input.shape}")
flt = nn.Flatten(start_dim=1)
flt(input).shape

# 4. The Loss

## 4.0 The metric
1. 以NLL作为metric
2. 理论模型的metric可以提前算好，读进来.
3. 注意这里的metric比较的是target还是target_5:
    - 建议使用target data而不是target_5，因为后者是为了方便NN的learning，前者才是真正的比较NLL

In [291]:
def cal_metric(Pi, Mu, Sigma, Duration, N_gaussians, vali_setting):

    NLL = torch.tensor(0.,device=device,requires_grad=True)
    # print("vali_setting: ",vali_setting.shape)   torch.Size([40, 4])
    # [id,bidincrement,bidfee,retail]
    for i in range(len(Pi)):

        # d = vali_setting[i,1]
        # b = vali_setting[i,2]
        # v = vali_setting[i,3]
        # T = int((v-b)/d)

        target = Duration[i,:,0]
        pi = Pi[i,:]
        m = torch.distributions.Normal(loc=Mu[i,:], scale=Sigma[i,:])

        # Drop padded data and Expanded to the same dim
        idx = torch.nonzero(target)
        target_nonzero = target[idx]
        # target_nonzero_2 = torch.repeat_interleave(target_nonzero.unsqueeze(dim=1), repeats=N_gaussians, dim=1).to(device)
        target_nonzero_2 = target_nonzero.repeat(1,N_gaussians).to(device)

        # loss_1 是高斯分布的概率密度value
        loss_1 = torch.exp(m.log_prob(target_nonzero_2))

        # loss_2 是MDN的概率密度value
        loss_2 = torch.sum(loss_1 * pi, dim=1)

        ## 两种截断的方式都ok：
        ## 方法一：在MDN的prob上设定最小值；
        # loss_3 = torch.clamp(loss_2,min=MIN_LOSS)
        # loss_4 = torch.log(loss_3)
        # # loss_sum = -torch.mean(loss_4) + loss_sum
        # loss_sum = -torch.sum(loss_4) + loss_sum

        ## 方法二：在prob of each sample后加一个safety数 MIN_LOSS
        # loss_2 shape: torch.Size([40, 1])
        loss_3 = -torch.log(loss_2+MIN_LOSS)
        # 除以idx的长度，平均到每个auction
        NLL = torch.sum(loss_3)/len(idx) + NLL

    # 求metric不需要除以batch_size，需要累积起来所有vali data的nll
    return NLL

## 4.1 初始版本
### 4.1.1 version 1 [FAILED]

In [ ]:
# # 当input的shape是[50,3]时，输出应该是50个GMM
# # 对这50个GMM看能生成什么output

def loss_fn(Pi,duration,m,N_gaussians):
    loss_list = []
    with torch.no_grad():
        # for each GMM
        for i in range(len(Pi)):
            target = duration[i,:]
            pi = Pi[i,:]

            # Drop padded data and Expanded to the same dim
            target_nonzero = target[torch.nonzero(target)].squeeze_()
            target_nonzero = torch.repeat_interleave(target_nonzero.unsqueeze(dim=1), repeats=N_gaussians, dim=1).to(device)
            # len_target = len(target_nonzero)       # The length of target data

            # loss_1 是高斯分布的概率密度value
            loss_1 = torch.exp(m[i].log_prob(target_nonzero))

            # loss_2 是MDN的概率密度value
            loss_2 = torch.sum(loss_1 * pi, dim=1)

            # loss_3 是值非0的概率密度value
            loss_3 = loss_2[torch.nonzero(loss_2)].view(-1,1)                # 再去掉所有的log(0)

            # 如果loss_2全是0则赋值为1e-40，否则赋值为loss的最小值
            MIN_LOSS = torch.min(loss_3) if torch.min(loss_2)>0 else 1e-40
            loss_padded = [MIN_LOSS]* (loss_2.shape[0]-loss_3.shape[0])
            loss_padded_ts = torch.tensor(loss_padded,device=device)

            # loss_4是用MIN_LOSS填充后的loss_3，和loss_2等长
            loss_4 = torch.cat((loss_3[:,0],loss_padded_ts))
            torch._assert(len(loss_4) == len(loss_2),"cat不正确")

            # loss_5是log likelihood loss
            loss_5 = torch.log(loss_4)
            loss_list.append(-torch.mean((loss_5)).item())
            # loss_list.append(-torch.sum((loss_5)).item()) # loss值会比较大，

    # 最后loss求一下平均
    loss = np.mean(loss_list)
    loss_ts = torch.tensor(loss,device=device,requires_grad=True)
    return loss_ts,loss_list

### 4.1.2 version 2 [WORK]

In [365]:
def loss_fn_v2(Pi, Mu, Sigma, Duration, N_gaussians):

    loss_sum = torch.tensor(0.,device=device,requires_grad=True)

    for i in range(len(Pi)):

        target = Duration[i,:,0]
        pi = Pi[i,:]
        m = torch.distributions.Normal(loc=Mu[i,:], scale=Sigma[i,:])

        with torch.no_grad():
            # Drop padded data and Expanded to the same dim
            idx = torch.nonzero(target)
            target_nonzero = target[idx]
            target_nonzero_2 = target_nonzero.repeat(1,N_gaussians).to(device)

        # loss_1 是高斯分布的概率密度value
        loss_1 = torch.exp(m.log_prob(target_nonzero_2))

        # loss_2 是MDN的概率密度value
        loss_2 = torch.sum(loss_1 * pi, dim=1)

        ## 两种截断的方式都ok：
        ## 方法一：在MDN的prob上设定最小值；
        # loss_3 = torch.clamp(loss_2,min=MIN_LOSS)
        # loss_4 = torch.log(loss_3)
        # # loss_sum = -torch.mean(loss_4) + loss_sum
        # loss_sum = -torch.sum(loss_4) + loss_sum

        SAFETY = 1e-20
        ## 方法二：在prob of each sample后加一个safety数 MIN_LOSS
        loss_3 = -torch.log(loss_2+SAFETY)
        loss_sum = torch.sum(loss_3) + loss_sum

    # 最后loss求一下平均
    return loss_sum/len(Pi)

### 4.1.3 version 3 [LogSumExp]
1. $ exponent_i = \log_{}{\alpha_i } -1/2*\log_{}{2\pi } -\log_{}{\sigma_i }- \frac{(x-\mu_i )^2}{2\sigma_i^2} $
2. $ NLL =-\sum_{j}^{N_S}LogSumExp= -\sum_{j}^{N_S} \log\sum_{i}^{N_G} \exp \{exponent_i\}$

In [366]:
def log_sum_exp(x):
    """Log-sum-exp trick implementation"""

    # x.shape: torch.Size([35, 3])
    # x_max: torch.Size([35, 1])
    # torch.max()[0]， 只返回最大值的每个数,[1]则返回下标

    x_max = torch.max(x, dim=1, keepdim=True)[0]
    x_min = torch.abs(torch.min(x, dim=1, keepdim=True)[0])
    x_mid = torch.median(x,dim=1,keepdim=True)[0]
    # x_min和x_max 通常都是负数

    # print("x_max:",x_max)
    # print("x_min:",x_min)
    return torch.log(torch.sum(torch.exp(x - x_max), dim=1, keepdim=True)) + x_max
    # return torch.log(torch.sum(torch.exp(x), dim=1, keepdim=True))

In [367]:
def loss_fn_v3(Pi, Mu, Sigma, Target, N_gaussians):

    loss_sum = torch.tensor(0.,device=device,requires_grad=True)

    for i in range(len(Pi)):

        target = Target[i,:,0]
        pi = Pi[i,:]
        sigma = Sigma[i,:]
        mu = Mu[i,:]

        # Drop padded data and Expanded to the same dim
        idx = torch.nonzero(target)
        target_nonzero = target[idx]
        target_nonzero_2 = target_nonzero.repeat(1,N_gaussians)

        # loss_1 是高斯分布的概率密度value
        # if(torch.isnan(torch.log(sigma).detach()).any()):
        #     print("ALERT!!!!!!!!!!!!!!!!!: torch.log(sigma)")
        # if(torch.isnan(torch.log(pi).detach()).any()):
        #     print("ALERT!!!!!!!!!!!!!!!!!: torch.log(pi)")
        v1 = .5 * torch.log(2 * torch.tensor(np.pi)) -torch.log(sigma)
        v2 = (target_nonzero_2 - mu)
        # v1 torch.Size([3])
        # v2 torch.Size([44, 3])
        exponent = torch.log(pi) - v1 - .5*(v2**2)/((sigma)**2)
        log_gauss = -log_sum_exp(exponent)

        # 在samples维度上继续sum一次
        loss_sum = torch.sum(log_gauss) + loss_sum

    loss_ts = loss_sum/len(Pi)
    return loss_ts

### 4.1.4 version 5-Laplace
1.version 2 但是是Laplace

In [368]:
def loss_fn_v5(Pi, Mu, Sigma, Duration, N_gaussians):

    loss_sum = torch.tensor(0.,device=device,requires_grad=True)

    for i in range(len(Pi)):

        target = Duration[i,:,0]
        pi = Pi[i,:]
        m = torch.distributions.Laplace(loc=Mu[i,:], scale=Sigma[i,:])

        # Drop padded data and Expanded to the same dim
        idx = torch.nonzero(target)
        target_nonzero = target[idx]
        # target_nonzero_2 = torch.repeat_interleave(target_nonzero.unsqueeze(dim=1), repeats=N_gaussians, dim=1).to(device)
        target_nonzero_2 = target_nonzero.repeat(1,N_gaussians).to(device)

        # loss_1 是高斯分布的概率密度value
        loss_1 = torch.exp(m.log_prob(target_nonzero_2))

        # loss_2 是MDN的概率密度value
        loss_2 = torch.sum(loss_1 * pi, dim=1)

        ## 两种截断的方式都ok：
        ## 方法一：在MDN的prob上设定最小值；
        loss_3 = torch.clamp(loss_2,min=MIN_LOSS)
        loss_4 = torch.log(loss_3)
        # loss_sum = -torch.mean(loss_4) + loss_sum
        loss_sum = -torch.sum(loss_4) + loss_sum

        ## 方法二：最后加一个safety数 MIN_LOSS
        # loss_3 = torch.log(loss_2+MIN_LOSS)
        # loss_sum = -torch.mean(loss_3) + loss_sum

    # 最后loss求一下平均
    return loss_sum/len(Pi)

## 4.2 cdf loss
1. 注意把targets data的路径改一下

In [ ]:
# cdf loss
###################
def loss_fn_cdf(Pi, Mu, Sigma, Target, N_gaussians):

    loss_sum = torch.tensor(0.,device=device,requires_grad=True)

    # for each GMM
    for i in range(len(Pi)):
        target = Target[i,:,0]
        prob_target = Target[i,:,1]
        pi = Pi[i,:]
        m = torch.distributions.Normal(loc=Mu[i,:], scale=Sigma[i,:])

        with torch.no_grad():
            # Drop padded data and Expanded to the same dim
            #### 写法一：work。无原地变换
            idx = torch.nonzero(target)
            target_nonzero = target[idx]
            prob_target_nonzero = torch.squeeze(prob_target[idx])

            target_nonzero_2 = target_nonzero.repeat(1,N_gaussians).to(device)

        # loss_1 是高斯分布的概率密度value
        loss_1 = torch.exp(m.cdf(target_nonzero_2))

        # loss_2 是MDN的概率密度value
        loss_2 = torch.sum(loss_1 * pi, dim=1)

        # loss_3 是cdf的差值的一范数or二范数，表示的是MDN和target之间的总体差异
        loss_3 = torch.norm((loss_2 - prob_target_nonzero), 2)

        # loss_sum = torch.log(loss_3) + loss_sum
        loss_sum = loss_3 + loss_sum

    # Batch平均loss
    return loss_sum/len(Pi)

## 4.3 交叉熵/ KL
1. 自己写的，没有用torch的函数

In [ ]:
# 当input的shape是[50,3]时，输出应该是50个GMM
def loss_preparation_CE(pi, mu, sigma):

    m=[]
    for i in range(pi.shape[0]):
        m.append(torch.distributions.Normal(loc=mu[i,:], scale=sigma[i,:]))

    # target_packed = pack_padded_sequence(target,target_len,batch_first=True,enforce_sorted=False)  # 去掉padded 0并拉成一个vector
    # seq_unpacked, lens_unpacked = pad_packed_sequence(target_packed, batch_first=True)             # 和上面互为逆运算

    return m

In [ ]:
# # 当input的shape是[50,3]时，输出应该是50个GMM
# # 对这50个GMM看能生成什么output

def loss_fn_CE(Pi, Mu, Sigma, Target, N_gaussians):

    loss_sum = torch.tensor(0.,device=device,requires_grad=True)

    # for each GMM
    for i in range(len(Pi)):
        target = Target[i,:,0]
        prob_target = Target[i,:,1]
        pi = Pi[i,:]
        m = torch.distributions.Normal(loc=Mu[i,:], scale=Sigma[i,:])

        # Drop padded data and Expanded to the same dim
        #### 写法一：work。无原地变换
        idx = torch.nonzero(target)
        target_nonzero = torch.squeeze(target[idx])
        prob_target_nonzero = torch.squeeze(prob_target[idx])

        target_nonzero_2 = torch.repeat_interleave(target_nonzero.unsqueeze(dim=1), repeats=N_gaussians, dim=1).to(device)

        # loss_1 是高斯分布的概率密度value
        loss_1 = torch.exp(m.log_prob(target_nonzero_2))

        # loss_2 是MDN的概率密度value
        loss_2 = torch.sum(loss_1 * pi, dim=1)

        # loss_4是KL loss
        # loss_4 = -prob_target_nonzero*torch.log(loss_2) + prob_target_nonzero*torch.log(prob_target_nonzero)
        # print("prob_target_nonzero shape:",prob_target_nonzero.shape)
        # print("loss_2 shape:",loss_2.shape)

        loss_4 = -prob_target_nonzero*torch.log(loss_2+MIN_LOSS)+ prob_target_nonzero*torch.log(prob_target_nonzero)
        loss_sum = torch.sum(loss_4)+loss_sum

    loss_ts = loss_sum/len(Pi)
    return loss_ts

In [ ]:
def loss_test():
    loss_2 = torch.tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    loss_3 = loss_2[torch.nonzero(loss_2)].view(-1,1)         # 去掉所有的log(0)
    print("loss_3:",loss_3.shape)

    MIN_LOSS = torch.min(loss_2) if torch.min(loss_2)>0 else 1e-20

    print("MIN_LOSS：",MIN_LOSS)
    # 用MIN_loss替代loss_2为0的部分
    loss_2[loss_2==0] = MIN_LOSS


    loss_padded = [MIN_LOSS]* (loss_2.shape[0]-loss_3.shape[0])
    loss_padded_ts = torch.tensor(loss_padded) # ,device=device
    loss_3 = torch.cat((loss_3[:,0],loss_padded_ts))
    loss_3.shape, torch.log(loss_3)
    print("data: ",loss_3.data)
    print("data: ",type(loss_3))
# loss_test()

## 4.5 W-distance
1. 拿到target data的两列，按照第一列去predict

In [ ]:
p = 2
entreg = .1 # entropy regularization factor for Sinkhorn
factor = 1  # prob的放大系数
# 若以欧式距离为metric，则cost function可以直接用geomloss提供的 Sinkhorn快速解
OTLoss = geomloss.SamplesLoss(
    loss='sinkhorn', p=p, blur=entreg**(1/p), backend='tensorized', verbose=True)

def loss_fn_WD(Pi, Mu, Sigma, Duration, N_gaussians):

    loss_sum = torch.tensor(0.,device=device,requires_grad=True)

    for i in range(len(Pi)):

        target = torch.unique(Duration[i],dim=0)
        n_target = target[:,0]
        non_zero_idx = torch.nonzero(n_target)
        n = n_target[non_zero_idx]
        p_target = target[:,1]
        p = p_target[non_zero_idx]*factor

        # 放大“p”
        y_target = torch.cat([n,p],dim=1)

        pi = Pi[i,:]
        m = torch.distributions.Normal(loc=Mu[i,:], scale=Sigma[i,:])
        x = torch.repeat_interleave(n, repeats=N_gaussians, dim=1)   # expand dim
        y = (m.cdf(x+TARGET) - m.cdf(x)).to(device=device)
        y_cdf = torch.sum(pi*y,dim=1).unsqueeze(dim=1)*factor
        y_pred = torch.cat([n,y_cdf],dim=1)

        loss_sum = OTLoss(y_pred,y_target) + loss_sum

    return loss_sum/len(Pi)

# 5 Plot
1. draw:
    - loss: training data的loss和test data的loss趋势
    - MLP的网络结构（.png）
    - target distrb.和pred. distrb.
    - 所有target data的分布图


In [ ]:
writer = SummaryWriter("logs-MLP")
viz = Visdom(env="001")
print("Done")

## 5.1 draw 所有的target data
1. 实际上在`i < 50`里决定画前50 or what target data

In [ ]:
def draw_all_target_data():
    # target data
    # target_path = r"../data/targets"
    target_path = r"../data/targets_5"
    # target_path里有全部的target data地址
    target_all_path = os.listdir(target_path)
    len_target = len(target_all_path)
    print(f"一共有 *{len_target}* 组 target data")

    # Init
    # viz_env_str = "target_bar"
    # viz_env_str = "target_line"
    # viz_env_str = "target_hist"
    viz_env_str = "target_scatter"
    viz = Visdom(env = viz_env_str)

    for i in range(len_target):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Init
        win_str = str(target_all_path[i])
        title_str = "Target Distrb. of "+win_str

        if(i < 50):
            # Hist plot which is not that great
            if viz_env_str == "target_hist" :
                viz.histogram(X = np.array(target_df.N), env=viz_env_str, win=win_str,
                            opts= dict(title=title_str,numbins = 50))

            # Line plot
            if viz_env_str == "target_line" :
                target_df.drop_duplicates(inplace=True)
                viz.line(X = np.array(target_df.N),Y= np.array(target_df.P), env=viz_env_str, win=win_str,opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

            # Bar plot. 这个看起来是最准确的一个
            if viz_env_str == "target_bar" :
                target_df.drop_duplicates(inplace=True)
                arr_str = [str(i) for i in np.array(target_df.N)]
                viz.bar(X=np.array(target_df.P), env=viz_env_str, win=win_str,
                        opts=dict(title=title_str,rownames=arr_str,stacked = False))

            # Scatter plot
            if viz_env_str == "target_scatter" :
                target_df.drop_duplicates(inplace=True)
                viz.scatter(X=np.array(target_df),env=viz_env_str, win=win_str,opts=dict(title=title_str,markersize = 3))
    print("Done")

# draw_all_target_data()

- 画target data的cdf

In [ ]:
def draw_all_target_data_cdf():
    # target data
    # target_path = r"../data/targets"
    target_path = r"../data/targets_5_cdf"
    # target_path里有全部的target data地址
    target_all_path = os.listdir(target_path)
    len_target = len(target_all_path)
    print(f"一共有 *{len_target}* 组 target data")

    # Init，只需要改这个str就ok
    # viz_env_str = "target_barCDF"
    # viz_env_str = "target_lineCDF"
    # viz_env_str = "target_histCDF"
    viz_env_str = "target_scatterCDF"
    viz = Visdom(env = viz_env_str)

    for i in range(len_target):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Init
        win_str = str(target_all_path[i])
        title_str = "Target Distrb. of "+win_str

        if(i < 50):
            # Hist plot which is not that great
            if viz_env_str == "target_histCDF" :
                viz.histogram(X = np.array(target_df.N), env=viz_env_str, win=win_str,
                            opts= dict(title=title_str,numbins = 50))

            # Line plot
            if viz_env_str == "target_lineCDF" :
                target_df.drop_duplicates(inplace=True)
                viz.line(X = np.array(target_df.N),Y= np.array(target_df.P), env=viz_env_str, win=win_str,opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

            # Bar plot. 这个看起来是最准确的一个
            if viz_env_str == "target_barCDF" :
                target_df.drop_duplicates(inplace=True)
                arr_str = [str(i) for i in np.array(target_df.N)]
                viz.bar(X=np.array(target_df.P), env=viz_env_str, win=win_str,
                        opts=dict(title=title_str,rownames=arr_str,stacked = False))

            # Scatter plot
            if viz_env_str == "target_scatterCDF" :
                target_df.drop_duplicates(inplace=True)
                viz.scatter(X=np.array(target_df),env=viz_env_str, win=win_str,opts=dict(title=title_str,markersize = 3))
    print("Done")

# draw_all_target_data_cdf()

- 也是画target data，但是是平滑处理
- `savgol_filter`参数解析:
    - y：代表曲线点坐标（x,y）中的y
    - window_length：窗口长度，该值需为正奇整数。值越小，曲线越贴近真实曲线；值越大，平滑效果越厉害
    - polyorder: 对窗口内的数据点进行k阶多项式拟合，k的值需要小于window_length。值越大，曲线越贴近真实曲线；值越小，平滑效果越厉害
    - mode：确定了要应用滤波器的填充信号的扩展类型。（This determines the type of extension to use for the padded signal to which the filter is applied. ）
- **实际上没用到这个函数**

In [ ]:
def draw_smooth_target_data():
    # target data
    target_path = r"../data/targets"
    # target_path里有全部的target data地址
    target_all_path = os.listdir(target_path)
    len_target = len(target_all_path)
    print(f"一共有 *{len_target}* 组 target data")

    # Init
    viz_env_str = "target_bar"
    # viz_env_str = "target_distrb"
    # viz_env_str = "target_hist"
    # viz_env_str = "target_scatter"
    viz = Visdom(env = viz_env_str)

    for i in range(len_target):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Init
        win_str = str(target_all_path[i])
        title_str = "Target Distrb. of "+win_str

        if(i < 100):
            # Smoothen the data
            y_smooth = scipy.signal.savgol_filter(target_df.P,window_length=53,polyorder=3)
            # y_smooth = scipy.signal.savgol_filter(target_df.P, 99, 1, mode= 'nearest')
            target_df.P = y_smooth

            # Hist plot which is not that great
            # viz.histogram(X = np.array(target_df.N), env=viz_env_str, win=win_str,
            #             opts= dict(title=title_str,numbins = 50))

            # Line plot
            # target_df.drop_duplicates(inplace=True)
            # viz.line(X = np.array(target_df.N),Y= np.array(target_df.P), env=viz_env_str, win=win_str,opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

            # Bar plot. 这个看起来是最准确的一个
            target_df.drop_duplicates(inplace=True)
            arr_str = [str(i) for i in np.array(target_df.N)]
            viz.bar(X=np.array(target_df.P), env=viz_env_str, win=win_str,
                    opts=dict(title=title_str,rownames=arr_str))

            # Scatter plot
            # target_df.drop_duplicates(inplace=True)
            # viz.scatter(X=np.array(target_df),env=viz_env_str, win=win_str,opts=dict(title=title_str,markersize = 3))
    print("Done")

# draw_smooth_target_data()

In [ ]:
#### Test for drawing
def test_draw():
    viz = Visdom(env="001")

    mu = torch.tensor([0,10,20])
    sigma = torch.tensor([1,1,1])
    duration = torch.tensor([0,1,2,0])
    duration = torch.repeat_interleave(duration.unsqueeze(dim=1), repeats=3, dim=1)
    m = torch.distributions.Normal(loc=mu, scale=sigma)
    pi = torch.tensor([0.2,0.3,0.5])

    # draw
    x_0 = torch.tensor(np.arange(0,1000))
    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=3, dim=1)
    y = torch.exp(m.log_prob(x))
    y_sum = torch.unsqueeze(torch.sum(pi*y,dim=1),dim=1)
    viz.line(X = x_0,Y= torch.cat([y,y_sum],dim = 1), env="001", win="test_draw_2",
            opts= dict(title='test_draw', legend=['N1', 'N2', 'N3','NNN']))
# test_draw()

## 5.2 plot mdn

### 5.2.1 mdn pdf的图像[with input data]
1. 求解[1,6,11...]上的cdf：
    - 求解[1,6,11..]的cdf，再求解[6,11,..]，两者相减

In [ ]:
def draw_mdn_2(pi, mu, sigma, target, input, total_train_step, N_gaussians):
    """
    画两条曲线，pred-1表示归一化前，pred-2表示归一化后

    :param pi:
    :param mu:
    :param sigma:
    :param target:
    :param input:
    :param total_train_step:
    :param N_gaussians:
    :return:
    """
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = int(max(n).item())

    # The predicted distrb.
    m = torch.distributions.Normal(mu,sigma)          # Gaussian
    # m = torch.distributions.Laplace(mu,sigma)           # Laplace

    x_0 = torch.arange(1,(max_n+TARGET),TARGET).to(device=device)

    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)   # expand dim
    # 求解每个长度为TARGET的区间上的cdf
    y = (m.cdf(x+TARGET) - m.cdf(x)).to(device=device)

    # 方法一：
    y_pred_1 = torch.sum(pi*y,dim=1)
    # 方法二：做一下(0,1)上的归一化
    y_pred_2 = y_pred_1/y_pred_1.sum()

    # The input distrb.
    input_data = input[0:2,:]
    x_input = torch.arange(1,(300+1),SCALE).to(device=device)
    y_input_0 = (input_data[0,:]).to(device=device)
    y_input_1 = (input_data[1,:]).to(device=device)

    # Init
    win_str = "total_train_step = "+str(total_train_step)
    title_str = "Distrb. in "+win_str
    viz.line(X = [0.],Y = [0.], env="001", win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env="001", win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred_1, env="001", win=win_str, update="append", name='pred-1',
            opts= dict(title=title_str))

    # Plot y_pred_2
    viz.line(X = x_0,Y= y_pred_2, env="001", win=win_str, update="append", name='pred-2',
            opts= dict(title=title_str))

    # Plot y_input
    # 如果input data长度短，全部画完
    if(len(x_input) <= max_n):
        viz.line(X = x_input,Y= y_input_0, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        viz.line(X = x_input,Y= y_input_1, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))
    # 如果input data长度长，则截断
    else:
        x_input_0 = x_input[0:max_n]
        y_input_2 = y_input_0[0:max_n]/y_input_0[0:max_n].sum()
        y_input_3 = y_input_1[0:max_n]/y_input_1[0:max_n].sum()
        viz.line(X = x_input_0,Y= y_input_2, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        viz.line(X = x_input_0,Y= y_input_3, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

### 5.2.2 plot mdn cdf[无input data]

In [ ]:
def draw_mdn_cdf(pi, mu, sigma, target, input, total_train_step, N_gaussians):
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = max(n).item()               # 横轴长度

    # The predicted distrb.
    m = torch.distributions.Normal(mu,sigma)
    x_0 = torch.arange(1,max_n).to(device=device)
    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)
    y = m.cdf(x).to(device=device)                        # y:多条高斯曲线; y_pred: 一条GMM曲线
    # y_pred = torch.unsqueeze(torch.sum(pi*y,dim=1),dim=1)                 # 维度相等才能cat
    y_pred = torch.sum(pi*y,dim=1)

    # Init
    win_str = "total_train_step = "+str(total_train_step)
    title_str = "Distrb. in "+win_str
    viz.line(X = [0.],Y = [0.], env="001", win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env="001", win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred, env="001", win=win_str, update="append", name='pred',
            opts= dict(title=title_str))

### 5.2.3 plot mdn pdf in TEST
1. 读入2个model时，在test set上比较效果

In [ ]:
test_env = "003"
def draw_mdn_3(Pi1, Mu1, Sigma1, Pi2, Mu2, Sigma2, Target, Input, test_batch, N_gaussians):
    for i in range(len(Pi1)):
        # The target distrb.
        target = Target[i]
        target = torch.unique(target,dim=0)    # drop the duplicate
        n_target = target[:,0]
        non_zero_idx = torch.nonzero(n_target)
        n = n_target[non_zero_idx]

        p_target = target[:,1]
        p = p_target[non_zero_idx]
        max_n = int(max(n).item())

        # The predicted distrb.
        mu1 = Mu1[i]
        mu2 = Mu2[i]
        sigma1 = Sigma1[i]
        sigma2 = Sigma2[i]
        m1 = torch.distributions.Normal(mu1,sigma1)          # Gaussian
        m2 = torch.distributions.Normal(mu2,sigma2)          # Gaussian
        # m = torch.distributions.Laplace(mu,sigma)           # Laplace

        x_0 = torch.arange(1,(max_n+TARGET),TARGET).to(device=device)

        x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)   # expand dim
        # 求解每个长度为TARGET的区间上的cdf
        y1 = (m1.cdf(x+TARGET) - m1.cdf(x)).to(device=device)
        y2 = (m2.cdf(x+TARGET) - m2.cdf(x)).to(device=device)

        # 方法一：
        pi1 = Pi1[i]
        pi2 = Pi2[i]
        y_mdn_1 = torch.sum(pi1*y1,dim=1)
        y_mdn_2 = torch.sum(pi2*y2,dim=1)
        # 方法二：做一下(0,1)上的归一化
        y_pred_1 = y_mdn_1/y_mdn_1.sum()
        y_pred_2 = y_mdn_2/y_mdn_2.sum()

        # The input distrb.
        input = Input[i]
        input_data = input[0:2,:]
        x_input = torch.arange(1,(300+1),SCALE).to(device=device)
        y_input_0 = (input_data[0,:]).to(device=device)
        y_input_1 = (input_data[1,:]).to(device=device)

        # Init
        win_str = "Test batch = "+str(test_batch)+" idx = "+str(i)
        title_str = "Distrb. of "+win_str
        viz.line(X = [0.],Y = [0.], env=test_env, win=win_str, opts= dict(title=title_str))

        # Visdom本身不能把hist和line画在一个window中
        # 如果想画一起只能是两条line
        # Plot y_target
        # viz.histogram(X = n, env="001", win=win_str,
        #         opts= dict(title=title_str))
        viz.line(X = n,Y= p, env=test_env, win=win_str, update="append", name='target',
                opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

        # Plot y_pred
        viz.line(X = x_0,Y= y_pred_1, env=test_env, win=win_str, update="append", name='pred-MLE',opts= dict(title=title_str))
        viz.line(X = x_0,Y= y_pred_2, env=test_env, win=win_str, update="append", name='pred-WD',
                opts= dict(title=title_str))

        # Plot y_input
        # 如果input data长度短，全部画完
        if(len(x_input) <= max_n):
            viz.line(X = x_input,Y= y_input_0, env=test_env, win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
            viz.line(X = x_input,Y= y_input_1, env=test_env, win=win_str, update="append", name='GT-2', opts= dict(title=title_str))
        # 如果input data长度长，则截断
        else:
            x_input_0 = x_input[0:max_n]
            y_input_2 = y_input_0[0:max_n]/y_input_0[0:max_n].sum()
            y_input_3 = y_input_1[0:max_n]/y_input_1[0:max_n].sum()
            viz.line(X = x_input_0,Y= y_input_2, env=test_env, win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
            viz.line(X = x_input_0,Y= y_input_3, env=test_env, win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

## 5.3 png格式的net structure

In [ ]:
def draw_the_net_png():

    x = torch.randn([5, 3, 300])  # 定义网络的输入值
    mlp = MLP(N_gaussians)
    y = mlp(x)                    # 获取网络的预测值

    MyConvNetVis = make_dot(y, params=dict(list(mlp.named_parameters()) + [('x', x)]))
    MyConvNetVis.format = "png"
    # 指定文件生成的文件夹
    MyConvNetVis.directory = "data_pic"
    # 生成文件
    MyConvNetVis.view()
# draw_the_net_png()

## 5.4 plot the loss
1. 直接根据参数决定draw什么loss，反正loss都画在一个window里面

In [ ]:
win_train_loss_str = "The Loss of BATCH in the Training Data"
win_vali_loss_str = "The Loss in the Vali Data"
win_train_epoch_loss_str = "The Loss of EPOCH in the Training Data"

def draw_loss(X_step, loss, win_str):
    viz.line(X = [X_step], Y = [loss],win=win_str, update="append",
        opts= dict(title=win_str))


## 5.5 plot the metric

In [ ]:
win_vali_metric_str = "The NLL of ALL vali data"
def draw_metric(X_step, total_vali_metric, GT_metric):

    viz.line(X = [X_step], Y = [[total_vali_metric,GT_metric[0,0],GT_metric[0,1]]],win=win_vali_metric_str, update="append", opts= dict(title=win_vali_metric_str, legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))


## 5.6 grad graph
https://github.com/t-vi/pytorch-tvmisc/blob/master/visualize/bad_grad_viz.ipynb

In [ ]:

def iter_graph(root, callback):
    queue = [root]
    seen = set()
    while queue:
        # 队头出fn然后判断fn是否seen
        fn = queue.pop()
        if fn in seen:
            continue
        # unseen加入queue，并且继续沿着fn向下递归（DFS？）
        seen.add(fn)
        for next_fn, _ in fn.next_functions:
            # Alert：只有非none的才会加入queue，然后在register_grad里被fn_dict记录下grad_input
            if next_fn is not None:
                queue.append(next_fn)
        callback(fn)

def register_hooks(var):
    fn_dict = {}  # 记录了grad名字和值（grad_input）
    def hook_cb(fn):
        def register_grad(grad_input, grad_output):
            fn_dict[fn] = grad_input
        fn.register_hook(register_grad)

    # 1. 递归注册grad_input
    iter_graph(var.grad_fn, hook_cb)

    # def is_bad_grad(grad_output):
    #     if grad_output is None:
    #         return False
    #     return grad_output.isnan().any() or (grad_output.abs() >= 1e6).any()
    def is_bad_grad(grad_output):
        if grad_output is None:
                return True
        grad_output = grad_output.data
        return grad_output.ne(grad_output).any() or grad_output.gt(1e6).any()

    # 2. graph已经从grad input构建完了，还有grad output的判断以及颜色的选择
    def make_dot():
        node_attr = dict(style='filled',
                        shape='box',
                        align='left',
                        fontsize='12',
                        ranksep='0.1',
                        height='0.2')
        dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))

        def size_to_str(size):
            return '('+(', ').join(map(str, size))+')'

        def build_graph(fn):
            if hasattr(fn, 'variable'):  # if GradAccumulator
                u = fn.variable
                node_name = 'Variable\n ' + size_to_str(u.size())
                dot.node(str(id(u)), node_name, fillcolor='lightblue')
            else:  # 不是variable属性，白色
                assert fn in fn_dict, fn   # 判断fn在不在fn_dict里面！
                fillcolor = 'white'
                if any(is_bad_grad(gi) for gi in fn_dict[fn]):
                    fillcolor = 'red'
                dot.node(str(id(fn)), str(type(fn).__name__), fillcolor=fillcolor)
            for next_fn, _ in fn.next_functions:
                if next_fn is not None:
                    next_id = id(getattr(next_fn, 'variable', next_fn))
                    dot.edge(str(next_id), str(id(fn)))
        iter_graph(var.grad_fn, build_graph)

        return dot

    return make_dot

# x = torch.randn(10, 10, requires_grad=True)
# y = torch.randn(10, 10, requires_grad=True)
#
# z = x / (y * 0)
# z = z.sum() * 2
# get_dot = register_hooks(z)


# with torch.autograd.detect_anomaly():
#     z.backward()
#     dot = get_dot()
#     dot.save('tmp.dot') # to get .dot
#     dot.render('tmp') # to get SVG
#     # dot # in Jupyter, you can just render the variable

# 6 Training
## 6.1 Init

In [369]:
# torch.autograd.set_detect_anomaly(True)

mlp = MLP_1_5(N_gaussians)

# Init the params
# # mlp = model_param_init(mlp)
# Init the vis
viz.line(X = [0.],Y = [0.], env="001", win=win_train_loss_str, opts= dict(title=win_train_loss_str))
viz.line(X = [0.],Y = [0.], env="001", win=win_train_epoch_loss_str, opts= dict(title=win_train_epoch_loss_str))
viz.line(X = [0.],Y = [[0.,0.,0.]], env="001", win=win_vali_metric_str, opts= dict(title=win_vali_metric_str,legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))

# Save the init params
# torch.save(mlp.state_dict(), 'mlp_init.pth')

# Load the saved model
# model_path_WD = "mlp_train=300_WD.pth"
# model_path_LSE = "mlp_train=300_LSE.pth"
# model_data = torch.load(model_path_LSE)
# mlp.load_state_dict(model_data)

mlp = mlp.to(device=device)
summary(mlp, (3,300))

############ learning rate strategy ############
# 1.
# Set different lr for params
# id: id() 函数返回对象的唯一标识符，标识符是一个整数。返回对象的内存地址。
mu_params = list(map(id, mlp.z_mu.parameters()))
base_params = filter(lambda p: id(p) not in mu_params, mlp.parameters())
params = [{'params': base_params},         # 如果对某个参数不指定学习率，就使用最外层的默认学习率
        {'params': mlp.z_mu.parameters(), 'lr': lr_for_mu}]

optimizer = torch.optim.Adam(params, lr=learning_rate)

# optimizer = torch.optim.Adagrad(params,lr=learning_rate,weight_decay = 1e-4)
# optimizer = torch.optim.Adagrad(params,lr=learning_rate)

# 2.
# Or set lr decay
# StepLR为步进，每step_size个epoch，lr*gamma
# scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.8)
# 下面这个比较好用
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,20], gamma=0.5, last_epoch=-1)

# # Set the hooks
# #mlp.conv1.register_forward_hook(hook_forward_fn)
# mlp.conv1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.linear1.register_forward_hook(hook_forward_fn)
# #mlp.linear1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.z_pi.register_forward_hook(hook_forward_fn)
# mlp.z_pi.register_full_backward_hook(hook_backward_fn_pi)
#
# #mlp.z_mu.register_forward_hook(hook_forward_fn)
# mlp.z_mu.register_full_backward_hook(hook_backward_fn_mu)
#
# mlp.z_sigma.register_forward_hook(hook_forward_fn)
# mlp.z_sigma.register_full_[2, 3, 300]backward_hook(hook_backward_fn_sigma)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 3, 300]               6
           Flatten-2                  [-1, 900]               0
            Linear-3                    [-1, 9]           8,109
       BatchNorm1d-4                    [-1, 9]              18
           Dropout-5                    [-1, 9]               0
         LeakyReLU-6                    [-1, 9]               0
            Linear-7                    [-1, 3]              30
           Softmax-8                    [-1, 3]               0
            Linear-9                    [-1, 3]              30
           Linear-10                    [-1, 3]              30
Total params: 8,223
Trainable params: 8,223
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.03
Estimated Total

## 6.2 grad check


In [370]:
# # grad check
# input_test = torch.randn((1,3,100), requires_grad=True,device=device)
# test_ans = gradcheck(mlp.to(device), input_test, eps=1e-3)  #, eps=1e-6, atol=1e-4
# print("Are the gradients correct: ", test_ans)

## 6.3 START HERE
1. 使用Laplace分布改：
    - loss function和draw

In [371]:
# filename = "../log_file.txt"
# f = open(filename,'w')
total_train_step = 0
EPOCH_NUM = 50

train_start_time = time.time()
mlp.train()

for epoch in range(EPOCH_NUM):
    # # Save the net structure
    # net_file_name = "net_before_epoch"+str(epoch)+".pth"
    # net_path = net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    epoch_train_loss = 0
    print(f"========== Now this is EPOCH {epoch} ==========")
    for batch_id,data in enumerate(train_loader):

        input_data, target, setting, target_metric = data
        # print(f"---- {batch_id} batch----")
        # Do the inference
        input_data = input_data.to(device)
        target = target.to(device)
        pi, mu, sigma = mlp(input_data)

        # Save the params
        # params = list(mlp.named_parameters())

        # Cal the MLE loss and draw the distrb.
        loss = loss_fn_v2(pi, mu, sigma, target, N_gaussians)
        # loss = loss_fn_v3(pi, mu, sigma, target, N_gaussians)
        # loss = loss_fn_v5(pi, mu, sigma, target, N_gaussians)

        # Cal the CDF loss and draw the distrb.
        # loss = loss_fn_cdf(pi, mu, sigma, target, N_gaussians)

        # Cal the CE or KL loss and draw the distrb.
        # loss  = loss_fn_CE(pi, mu, sigma, target, N_gaussians)

        # Cal the WD loss and draw the distrb.
        # loss  = loss_fn_WD(pi, mu, sigma, target, N_gaussians)

        epoch_train_loss += loss.item()
        draw_loss(total_train_step, loss.item(),win_train_loss_str)

        # print("训练次数：{}，Loss：{}".format(total_train_step, loss.item()))

        # Optim
        optimizer.zero_grad()
        loss.backward()

        ########### before step() ###############

        # for name, parms in mlp.named_parameters():
        #     print("=====before step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        ######

        # 反向传播时检测是否有异常值，定位code
        # with torch.autograd.detect_anomaly():
        #     loss.backward()

        ######
        # get_dot = register_hooks(loss)
        # dot = get_dot()
        # dot.save('tmp.dot')   # to get .dot
        # dot.render('tmp')     # to get pdf

        ######
        # Print grad check
        # v_n = []    # name
        # v_v = []    # value
        # v_g = []    # grad
        # for name, parameter in mlp.named_parameters():
        #     v_n.append(name)
        #     v_v.append(parameter.detach().cpu().numpy() if parameter is not None else [0])
        #     v_g.append(parameter.grad.detach().cpu().numpy() if parameter.grad is not None else [0])
        # for i in range(len(v_n)):
        #     # if np.max(v_v[i]).item() - np.min(v_v[i]).item() < 1e-6:
        #     #     color = bcolors.FAIL + '*'
        #     if np.isnan(v_g[i]).any() or np.isnan(v_v[i]).any():
        #         color = bcolors.FAIL + '*'
        #     else:
        #         color = bcolors.OKGREEN + ' '
        #     print('%svalue %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_v[i]).item(), np.max(v_v[i]).item()))
        #     print('%sgrad  %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_g[i]).item(), np.max(v_g[i]).item()))

        ######
        optimizer.step()
        scheduler.step()

        # for name, parms in mlp.named_parameters():
        #     print("=====After step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        # Only draw the 1st result in a training batch (5 in total)
        # if total_train_step % 20 == 0:
        #     with torch.no_grad():
        #         draw_mdn_2(pi[0,:].clone().detach(), mu[0,:].clone().detach(), sigma[0,:].clone().detach(), target[0].clone().detach(), input_data[0].clone().detach(),total_train_step, N_gaussians)
        #         # draw_mdn_cdf(pi[0,:].detach(), m[0], target[0].detach(), total_train_step, loss_list[0],N_gaussians)
        total_train_step += 1


    ########### Do validation
    with torch.no_grad():
        mlp.eval()
        total_vali_metric = 0
        GT_metric = torch.tensor([0.,0.]).reshape(1,2)

        for vali_batch_id, vali_data in enumerate(val_loader):
            vali_input_data, vali_target, vali_setting , vali_metric = vali_data
            vali_input_data = vali_input_data.to(device)
            vali_target = vali_target.to(device)
            # vali_metric = vali_metric

            vali_pi, vali_mu, vali_sigma = mlp(vali_input_data)

            # Compute the error/ metric
            vali_nll = cal_metric(vali_pi, vali_mu, vali_sigma, vali_target, N_gaussians, vali_setting)
            total_vali_metric += vali_nll

            # Sum up NLL of all vali data
            GT_metric += torch.sum(vali_metric,dim=0)
        # total_vali_metric = total_vali_metric/len(val_idx)

        # Get metric of GT model
        GT_metric = GT_metric/len(val_idx)
        total_vali_metric = total_vali_metric/len(val_idx)

        draw_metric(epoch, total_vali_metric.cpu(), GT_metric)
        mlp.train()

    # Plot the loss in this EPOCH
    print(f"========== IN EPOCH {epoch} the total loss is {epoch_train_loss} ==========")

    # # Save the net structure
    # net_file_name = "net_after_epoch"+str(epoch)+".pth"
    # net_path = net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    # Plot loss of this EPOCH
    draw_loss(epoch, epoch_train_loss,win_train_epoch_loss_str)
    # writer.add_scalars("Loss of EPOCH",{"Train":epoch_train_loss},epoch)

    # Record the params
    # for name,param in params:
    #     #参数的梯度
    #     #print("name:",name)
    #     #print("param.grad:",param.grad)
    #     # writer.add_histogram(tag = name +'_grad of EPOCH '+str(epoch),values=param.grad,global_step=epoch)
    #     # #参数值
    #     writer.add_histogram(tag = name +'data of EPOCH '+str(epoch),values=param.data,global_step=epoch)

# f.close()
train_end_time = time.time()
print(f"Total training time when epoch= *{EPOCH_NUM}* is *{train_end_time-train_start_time} *s")
writer.close()
print("Done")

========== Now this is EPOCH 0 ==========
========== IN EPOCH 0 the total loss is 17645.664154052734 ==========
========== Now this is EPOCH 1 ==========
========== IN EPOCH 1 the total loss is 14537.146301269531 ==========
========== Now this is EPOCH 2 ==========
========== IN EPOCH 2 the total loss is 14625.306365966797 ==========
========== Now this is EPOCH 3 ==========
========== IN EPOCH 3 the total loss is 14423.18472290039 ==========
========== Now this is EPOCH 4 ==========
========== IN EPOCH 4 the total loss is 14224.938842773438 ==========
========== Now this is EPOCH 5 ==========
========== IN EPOCH 5 the total loss is 13933.966491699219 ==========
========== Now this is EPOCH 6 ==========
========== IN EPOCH 6 the total loss is 13972.314239501953 ==========
========== Now this is EPOCH 7 ==========
========== IN EPOCH 7 the total loss is 14410.543884277344 ==========
========== Now this is EPOCH 8 ==========
========== IN EPOCH 8 the total loss is 14280.940460205078 ====

- save

In [ ]:
model_path_MLE = "AAAmlp_train=300_MLE.pth"
# model_path_WD = "mlp_train=300_WD_epoch=80.pth"
model_path_WD = "mlp_train=300_WD.pth"
model_path_LSE = "mlp_train=300_LSE.pth"
torch.save(mlp.state_dict(), model_path_MLE)
# torch.save(mlp.state_dict(), model_path_MLE)

# viz.delete_env("001_test")


In [ ]:
# Gtmp = pgv.AGraph('tmp.dot')
# G = nx.Graph(Gtmp)
# nx.draw(G)
# # plt.show()


$\mathcal{L}(y \vert x) = - \log\bigg\{\sum_{k=1}^K \pi_k(x)  \mathcal{N}\big(y \vert \mu_k(x), \Sigma_k(x)\big)\bigg\}$

# 7. Load and test
## 7.1 small batch
- 对某个小batch画图，batch size可以=1

In [ ]:
# Prepare the data
small_batch_size = 1

small_test_idx = train_idx[3:4]
small_test_loader = DataLoader(dataset = dataset,batch_size = small_batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(small_test_idx),collate_fn = my_collate_fn)
small_test_idx

In [ ]:
# Plot
test_env = "002"
def draw_mdn_test(pi, m, target, input, epoch, idx,N_gaussians,flag):
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = max(n).item()               # 横轴长度

    # The predicted distrb.
    x_0 = torch.arange(1,max_n).to(device=device)
    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)
    y = torch.exp(m.log_prob(x)).to(device=device)                          # y:多条高斯曲线; y_pred: 一条GMM曲线
    # y_pred = torch.unsqueeze(torch.sum(pi*y,dim=1),dim=1)                 # 维度相等才能cat
    y_pred = torch.sum(pi*y,dim=1)

    # The input distrb.
    input_data = input[0:2,:]
    x_input = torch.arange(1,input_data.shape[1]+1).to(device=device)
    y_input_0 = input_data[0,:].to(device=device)
    y_input_1 = input_data[1,:].to(device=device)

    # Init
    win_str = flag+ " epoch="+str(epoch)+" | idx="+str(idx)
    title_str = "Distrb. "+win_str
    viz.line(X = [0.],Y = [0.], env=test_env, win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env=test_env, win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred, env=test_env, win=win_str, update="append", name='pred',
            opts= dict(title=title_str))

    # Plot y_input
    viz.line(X = x_input,Y= y_input_0, env=test_env, win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
    viz.line(X = x_input,Y= y_input_1, env=test_env, win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

In [ ]:

for epoch in range(EPOCH_NUM):

    net_file_name_0 = "net_before_epoch"+str(epoch)+".pth"
    net_file_name_1 = "net_after_epoch"+str(epoch)+".pth"
    net_path0 = net_root_path + net_file_name_0
    net_path1 = net_root_path + net_file_name_1

    # Load the model
    mlp0 = MLP(N_gaussians)
    mlp1 = MLP(N_gaussians)
    mlp0.load_state_dict(torch.load(net_path0))
    mlp1.load_state_dict(torch.load(net_path1))
    mlp0.to(device)
    mlp1.to(device)

    # 其实每个epoch只会执行一个batch
    for batch_id,data in enumerate(small_test_loader):

        # Predict
        input_data, target = data
        input_data = input_data.to(device)
        target = target.to(device)
        pi0, mu0, sigma0 = mlp0(input_data)
        pi1, mu1, sigma1 = mlp1(input_data)
        # Get the mdn
        _,m0  = loss_preparation(pi0.detach(), mu0.detach(), sigma0.detach(), target.detach())
        _,m1  = loss_preparation(pi1.detach(), mu1.detach(), sigma1.detach(), target.detach())

        # Plot for 'small_batch_size' times
        for j in range(small_batch_size):
            draw_mdn_test(pi0[j,:].detach(), m0[j], target[j].detach(), input_data[j].detach(), epoch, j,N_gaussians,flag="before")
            draw_mdn_test(pi1[j,:].detach(), m1[j], target[j].detach(), input_data[j].detach(), epoch, j,N_gaussians,flag="after")

print("Done")

## 7.2 Test dataset

In [ ]:
# Load the saved model
# model_path_WD_2 = "mlp_train=300_WD_epoch=80.pth"
model_path_WD = "mlp_train=300_WD.pth"
model_path_MLE = "mlp_train=300_MLE.pth"


mlp_1 = MLP_1_3(N_gaussians)
mlp_2 = MLP_1_3(N_gaussians)

model_data_1 = torch.load(model_path_MLE)
# model_data_1 = torch.load(model_path_WD_2)
model_data_2 = torch.load(model_path_WD)

mlp_1.load_state_dict(model_data_1)
mlp_2.load_state_dict(model_data_2)

mlp_1.to(device)
mlp_2.to(device)

In [ ]:
total_test_loss_1 = 0
total_test_loss_2 = 0
i = 0

with torch.no_grad():
    for batch_id,data in enumerate(test_loader):
            input_data, target = data
            input_data = input_data.to(device)
            target = target.to(device)

            pi_1, mu_1, sigma_1 = mlp_1(input_data)
            pi_2, mu_2, sigma_2 = mlp_2(input_data)

            draw_mdn_3(pi_1, mu_1, sigma_1, pi_2, mu_2, sigma_2, target, input_data,i, N_gaussians)

            loss_1 = loss_fn_v2(pi_1, mu_1, sigma_1, target, N_gaussians)
            loss_2 = loss_fn_v2(pi_2, mu_2, sigma_2, target, N_gaussians)

            total_test_loss_1 += loss_1.item()
            total_test_loss_2 += loss_2.item()

            i += 1
    # 这里print的loss可以视作平均到每个sample上的loss
    print("total_test_loss_1 MLE:",total_test_loss_1/len(test_loader))
    print("total_test_loss_2 WD :",total_test_loss_2/len(test_loader))